In [2]:
! git clone https://github.com/zihangdai/xlnet

! pip install tf_sentencepiece
! pip install sentencepiece

! wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
! unzip /content/cased_L-24_H-1024_A-16.zip
! rm /content/cased_L-24_H-1024_A-16.zip

Cloning into 'xlnet'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (107/107), 129.24 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (51/51), done.
--2019-07-09 19:52:37--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip’

cased_L-24_H-1024_A 100%[===================>]   1.25G   114MB/s    in 9.3s    

2019-07-09 19:52:46 (138 MB/s) - ‘cased_L-24_H-1024_A-16.zip’ saved [1338042341/1338042341]

Archive:  /content/cased_L-24_H-1024_A-16.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index  
  in

In [0]:
import tensorflow as tf

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import numpy as np
import pandas as pd
from tqdm import tqdm
import collections
import os
import sys
import pprint
import json

import sentencepiece as spm

sys.path.insert(0, '/content/xlnet')

from run_classifier import FLAGS
FLAGS([None])

import function_builder
import model_utils
from run_classifier import InputExample
from run_classifier import file_based_convert_examples_to_features
from run_classifier import file_based_input_fn_builder
from prepro_utils import preprocess_text, encode_ids
from classifier_utils import convert_single_example, PaddingInputExample

tf.logging.set_verbosity(tf.logging.INFO)

def clear_checkpoint_cache():
  '''remove all files from FLAGS.model_dir'''
  files = tf.gfile.ListDirectory(FLAGS.model_dir)
  for file in files:
    tf.gfile.Remove(os.path.join(FLAGS.model_dir, file))
        
def get_lens():
  '''retrieve #_records for train & eval datasets'''
  with open('./dataset_lens.json') as file:
    lens = json.load(file)
    ret = lens['train'], lens['val']
  return ret

In [0]:
from google.colab import auth
auth.authenticate_user()

In [31]:
tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

with tf.Session(tpu_address) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4624636520845270748),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5862292550072502558),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8113030125432651549),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14384874521711867188),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1112809580184861644),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 3079818582861333513),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14918201319546221024),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11325530939983350382),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9260931485257681598),
 _DeviceAttributes(/job:tpu_wor

In [0]:
MODEL_SPEC = 'xlnet_cased_L-24_H-1024_A-16'
BUCKET_NAME = _________

gcs_path = 'gs://' + BUCKET_NAME
pretrained_model_path = gcs_path + '/' + MODEL_SPEC

TRAIN_TFREC_FILE_ = 'train.tf_record'
VAL_TFREC_FILE_ = 'val.tf_record'
TEST_TFREC_FILE_ = 'test.tf_record'

LABEL_LIST = ["0", "1"]

FLAGS.task_name = 'xlnet_test'
FLAGS.use_tpu = True
FLAGS.num_hosts = 1            # single TPU for colab
FLAGS.num_core_per_host = 8    # 8 cores per cloud TPU v2
FLAGS.tpu = tpu_address
FLAGS.overwrite_data = True
FLAGS.max_seq_length = 512

FLAGS.dropout = 0.1
FLAGS.dropatt = 0.1

FLAGS.learning_rate = 2e-6
FLAGS.train_batch_size = 16
FLAGS.train_steps = 10000
FLAGS.warmup_steps = 800
FLAGS.weight_decay = 0.00
FLAGS.decay_method = 'poly'    # ['poly', 'cos']

FLAGS.eval_batch_size = 128
FLAGS.predict_batch_size = 128

FLAGS.iterations = 1000        # iterations per loop, NOT total train steps
FLAGS.save_steps = 2000        # >= iterations
FLAGS.eval_all_ckpt = False
MAX_SAVED_CHECKPOINTS = 3

FLAGS.model_dir = gcs_path + '/model'    # checkpoints created during finetuning
FLAGS.output_dir = gcs_path + '/output'  # tf_record files (inputs)
FLAGS.predict_dir = gcs_path + '/predictions'
FLAGS.spiece_model_file = './' + MODEL_SPEC + '/spiece.model' # local FS
FLAGS.model_config_path = pretrained_model_path + '/xlnet_config.json' # must be on GCS
FLAGS.init_checkpoint = pretrained_model_path + '/xlnet_model.ckpt' # must be on GCS

train_tf_rec_file = FLAGS.output_dir + '/' + TRAIN_TFREC_FILE_
val_tf_rec_file = FLAGS.output_dir + '/' + VAL_TFREC_FILE_
test_tf_rec_file = FLAGS.output_dir + '/' + TEST_TFREC_FILE_

In [0]:
# ONE TIME PER MODEL SPEC - move pretrained model to GS
model_files = os.listdir('./' + MODEL_SPEC)
for file in model_files:
  os.system('gsutil cp ./' + MODEL_SPEC + '/' + file + ' ' + pretrained_model_path)

In [0]:
# ONE TIME PER FEATURE SET - import data and gen features

def load_data():
  # ... ...
  return train_df, val_df

train_df, val_df = load_data()

print(f'train records: {len(train_df)} val records: {len(val_df)}')

train records: 1325501 val records: 331376


In [ ]:
# ONE TIME PER FEATURE SET: encode inputs & convert to tf_record
#
# In the original xlnet.run_classifier.py, inputs are converted to features, then 
# saved as tf_record files every time a train or eval routine is initiated.
# If reusing features already cached to output_dir, skip this processing step.
#
# Length of each dataset is required for computing eval steps, so lengths
# are persisted to local FS as 'dataset_lens.json'.

sp = spm.SentencePieceProcessor()
sp.Load(FLAGS.spiece_model_file)

def tokenize_fn(text):
  text = preprocess_text(text, lower=False)
  return encode_ids(sp, text)

def create_tf_rec_inputs(df_in, file_out, type=None):
  assert type in ['train', 'val', 'test']
  examples = []
  for (idx, row) in tqdm(df_in.iterrows(), total=len(df_in), mininterval=10):

    if type in ['train', 'val']:
      example = InputExample(idx, row.comment_text, label=row.label)
    elif type == 'test':
      example = InputExample(idx, row.comment_text)

    examples.append(example)

  if type == 'train':
    np.random.shuffle(examples)
  
  if type == 'val':
    # eval examples must be padded to multiple of batch size
    while len(examples) % FLAGS.eval_batch_size != 0:
      examples.append(PaddingInputExample())
    assert len(examples) % FLAGS.eval_batch_size == 0
        
  file_based_convert_examples_to_features(examples,
                                          LABEL_LIST,
                                          FLAGS.max_seq_length,
                                          tokenize_fn,
                                          file_out, 1)

  return len(examples)

train_len = create_tf_rec_inputs(train_df, train_tf_rec_file, type='train')
val_len = create_tf_rec_inputs(val_df, val_tf_rec_file, type='val')
#create_tf_rec_inputs(test_df, test_tf_rec_file, type='test')

lens = {'train': train_len,
       'val': val_len}
with open('./dataset_lens.json', 'w') as file:
  json.dump(lens, file)  

In [ ]:
! gsutil ls $pretrained_model_path

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_address)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=FLAGS.model_dir,
    save_checkpoints_steps=FLAGS.save_steps,
    keep_checkpoint_max=MAX_SAVED_CHECKPOINTS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations,
        num_shards=FLAGS.num_hosts * FLAGS.num_core_per_host,
        per_host_input_for_training=is_per_host))

In [0]:
# from xlnet.run_classifier.py -- added AUC to metric_fn.

def get_model_fn(n_class):
  def model_fn(features, labels, mode, params):
    #### Training or Evaluation
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    #### Get loss from inputs
    if FLAGS.is_regression:
      (total_loss, per_example_loss, logits
          ) = function_builder.get_regression_loss(FLAGS, features, is_training)
    else:
      (total_loss, per_example_loss, logits
          ) = function_builder.get_classification_loss(
          FLAGS, features, n_class, is_training)

    #### Check model parameters
    num_params = sum([np.prod(v.shape) for v in tf.trainable_variables()])
    tf.logging.info('#params: {}'.format(num_params))

    #### load pretrained models
    scaffold_fn = model_utils.init_from_checkpoint(FLAGS)

    #### Evaluation mode
    if mode == tf.estimator.ModeKeys.EVAL:
      assert FLAGS.num_hosts == 1

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        eval_input_dict = {
            'labels': label_ids,
            'predictions': predictions,
            'weights': is_real_example
        }
        accuracy = tf.metrics.accuracy(**eval_input_dict)

        auc = tf.metrics.auc(**eval_input_dict)
        
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        return {
            'eval_accuracy': accuracy,
            'eval_auc': auc,
            'eval_loss': loss}

      def regression_metric_fn(
          per_example_loss, label_ids, logits, is_real_example):
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        pearsonr = tf.contrib.metrics.streaming_pearson_correlation(
            logits, label_ids, weights=is_real_example)
        return {'eval_loss': loss, 'eval_pearsonr': pearsonr}

      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)

      #### Constucting evaluation TPUEstimatorSpec with new cache.
      label_ids = tf.reshape(features['label_ids'], [-1])

      if FLAGS.is_regression:
        metric_fn = regression_metric_fn
      else:
        metric_fn = metric_fn
      metric_args = [per_example_loss, label_ids, logits, is_real_example]

      if FLAGS.use_tpu:
        eval_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode,
            loss=total_loss,
            eval_metrics=(metric_fn, metric_args),
            scaffold_fn=scaffold_fn)
      else:
        eval_spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=total_loss,
            eval_metric_ops=metric_fn(*metric_args))

      return eval_spec

    elif mode == tf.estimator.ModeKeys.PREDICT:
      label_ids = tf.reshape(features["label_ids"], [-1])

      predictions = {
          "logits": logits,
          "labels": label_ids,
          "is_real": features["is_real_example"]
      }

      if FLAGS.use_tpu:
        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
      else:
        output_spec = tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions)
      return output_spec

    #### Configuring the optimizer
    train_op, learning_rate, _ = model_utils.get_train_op(FLAGS, total_loss)

    monitor_dict = {}
    monitor_dict["lr"] = learning_rate

    #### Constucting training TPUEstimatorSpec with new cache.
    if FLAGS.use_tpu:
      #### Creating host calls
      if not FLAGS.is_regression:
        label_ids = tf.reshape(features['label_ids'], [-1])
        predictions = tf.argmax(logits, axis=-1, output_type=label_ids.dtype)
        is_correct = tf.equal(predictions, label_ids)
        accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

        monitor_dict["accuracy"] = accuracy

        host_call = function_builder.construct_scalar_host_call(
            monitor_dict=monitor_dict,
            model_dir=FLAGS.model_dir,
            prefix="train/",
            reduce_fn=tf.reduce_mean)
      else:
        host_call = None

      train_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op, host_call=host_call,
          scaffold_fn=scaffold_fn)
    else:
      train_spec = tf.estimator.EstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op)

    return train_spec

  return model_fn


model_fn = get_model_fn(len(LABEL_LIST) if LABEL_LIST is not None else None)

In [ ]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    predict_batch_size=FLAGS.predict_batch_size,
    eval_batch_size=FLAGS.eval_batch_size)

In [ ]:
train_input_fn = file_based_input_fn_builder(
  input_file=train_tf_rec_file,
  seq_length=FLAGS.max_seq_length,
  is_training=True,
  drop_remainder=True)

train_len, val_len = get_lens()
print('train len', train_len, 'val len', val_len)

In [0]:
clear_checkpoint_cache()
tf.gfile.ListDirectory(FLAGS.model_dir)

In [ ]:
FLAGS.train_steps = 10000
estimator.train(input_fn=train_input_fn, max_steps=FLAGS.train_steps)

In [ ]:
eval_input_fn = file_based_input_fn_builder(
    input_file=val_tf_rec_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=True)

In [ ]:
steps_and_files = []
filenames = tf.gfile.ListDirectory(FLAGS.model_dir)

for filename in filenames:
  if filename.endswith(".index"):
    ckpt_name = filename[:-6]
    cur_filename = os.path.join(FLAGS.model_dir, ckpt_name)
    global_step = int(cur_filename.split("-")[-1])
    tf.logging.info("Add {} to eval list.".format(cur_filename))
    steps_and_files.append([global_step, cur_filename])
steps_and_files = sorted(steps_and_files, key=lambda x: x[0])

if not FLAGS.eval_all_ckpt:
  steps_and_files = steps_and_files[-1:]

In [ ]:
eval_results = []
_, val_len = get_lens()
# for tpu evaluation, must pass the number of eval steps
eval_steps = val_len // FLAGS.eval_batch_size
print('eval steps', eval_steps)

for global_step, filename in sorted(steps_and_files, key=lambda x: x[0]):
  ret = estimator.evaluate(
      input_fn=eval_input_fn,
      steps=eval_steps,
      checkpoint_path=filename)
  
  ret["step"] = global_step
  ret["path"] = filename

  eval_results.append(ret)

  tf.logging.info("=" * 80)
  log_str = "Eval result | "
  for key, val in sorted(ret.items(), key=lambda x: x[0]):
    log_str += "{} {} | ".format(key, val)
  tf.logging.info(log_str)

In [46]:
eval_results

[{'eval_accuracy': 0.93880713,
  'eval_auc': 0.9182491,
  'eval_loss': 0.25729603,
  'global_step': 10000,
  'loss': 0.26374766,
  'path': 'gs://xlnet422/model/model.ckpt-10000',
  'step': 10000}]